In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import torch
from tensorflow import keras
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

## the process of creating class-prompts

---
##### # Load the CSV data
##### csv_file_path = '/kaggle/input/brain-dead-multimodal-data-for-hateful-meme/hateful_memes/hateful_memes_original.csv' 
##### data = pd.read_csv(csv_file_path)

##### data['prompt'] = data['img']
##### print(data.head())

---
##### from tensorflow.keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
##### from tensorflow.keras.preprocessing import image

##### # Load the ResNet-50 model pre-trained on ImageNet weights
##### model = ResNet50(weights='imagenet')
##### model = model.to(device)


---
##### def preprocess_image(img_path):
#####    img = image.load_img(img_path, target_size=(224, 224))
#####     img = image.img_to_array(img)
#####     img = np.expand_dims(img, axis=0)
#####     img = preprocess_input(img)
    
#####     # Make a copy of the array to ensure no negative strides
#####     img_copy = img.copy()
    
#####    # Convert the copy to a PyTorch tensor
#####     img_tensor = torch.tensor(img_copy)
#####     img_tensor = img_tensor.to(device)
    
#####     return img_tensor

##### def classify_and_caption(img_path):
#####     img = preprocess_image(img_path)
    
#####     preds = model.predict(img , verbose=0)
    
#####     decoded_preds = decode_predictions(preds, top=10)[0]  
#####     high_conf_preds = [f"{label} " for (_, label, prob) in decoded_preds if prob > 0.3]
    
#####     caption = ", ".join(high_conf_preds) if high_conf_preds else "messy"
    
#####     return caption

---
##### image_captions = []
##### i = 0
##### for img_path in data['img']:
#####    img_path2 = os.path.join('/kaggle/input/brain-dead-multimodal-data-for-hateful-meme/hateful_memes', img_path)
#####     caption = classify_and_caption(img_path2)
#####     i+=1
#####     print(i,end=",")
#####     if i % 100 == 0 : 
#####         print() 
#####     image_captions.append(caption)

---
##### image_captions = pd.DataFrame(image_captions)
##### data['prompt'] = image_captions 

---
##### data.to_csv( '/kaggle/working/one152.csv', index=False)

## loading the data where we generated the class-prompts using resnet152

In [30]:
# Load the CSV data
csv_file_path = '/kaggle/input/text-classification/one152.csv' 
data = pd.read_csv(csv_file_path)

print(data.head())

                                                text            img  label  \
0           a school bus that was engulfed in flames  img/32674.png      0   
1  when you ask your dad who is a retired drill s...  img/10246.png      1   
2    how i see kim burell everytime she grabs a mic!  img/14570.png      1   
3                           doing o's with the smoke  img/05316.png      0   
4                im gettin white girl wasted tonight  img/20936.png      0   

           prompt  
0        volcano   
1         prison   
2        gorilla   
3          cloak   
4  jigsaw_puzzle   


## label 1 means harmful content and 0 means normal

In [31]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model_path = '/kaggle/input/text-classification/resnet152 _ roberta .pth'
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


model.load_state_dict(torch.load(model_path))
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [32]:
for layer in model.distilbert.embeddings.parameters():
    layer.requires_grad = False

for layer in model.distilbert.transformer.layer[:3].parameters():
    layer.requires_grad = False

# Unfreeze the pre_classifier and classifier layers
for layer in model.pre_classifier.parameters():
    layer.requires_grad = True

for layer in model.classifier.parameters():
    layer.requires_grad = True

In [33]:
data['text'] = data['text'] + ' , ' + data['prompt']
data.drop(columns=['prompt'], inplace=True)
data

,text,img,label
0,"a school bus that was engulfed in flames , vol...",img/32674.png,0
1,when you ask your dad who is a retired drill s...,img/10246.png,1
2,how i see kim burell everytime she grabs a mic...,img/14570.png,1
3,"doing o's with the smoke , cloak",img/05316.png,0
4,"im gettin white girl wasted tonight , jigsaw_p...",img/20936.png,0
...,...,...,...
9995,when your grandma sees you for the first time ...,img/02631.png,0
9996,"i now declare you sir faggot sucker of cocks ,...",img/95712.png,1
9997,"help refugees , suit",img/41268.png,0
9998,"jolo jew only live once , sunglass",img/28076.png,0


In [34]:
max_length = 90  # Adjust as needed
tokens = tokenizer.batch_encode_plus(
    data['text'].tolist(),
    max_length=max_length,  
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
tokens

{'input_ids': tensor([[ 101, 1037, 2082,  ...,    0,    0,    0],
        [ 101, 2043, 2017,  ...,    0,    0,    0],
        [ 101, 2129, 1045,  ...,    0,    0,    0],
        ...,
        [ 101, 2393, 8711,  ...,    0,    0,    0],
        [ 101, 8183, 4135,  ...,    0,    0,    0],
        [ 101, 2045, 1005,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [35]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")
    
model = model.to(device)


CUDA is available. Using GPU.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
input_ids = torch.tensor(tokens['input_ids'].tolist())
attention_mask = torch.tensor(tokens['attention_mask'])
labels = torch.tensor(data['label'].tolist())

train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42)

train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)


train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=500, shuffle=False)

In [37]:
import torch.optim as optim
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [42]:
best_validation_loss = float('inf')  # Initialize the best validation loss to infinity

for epoch in range(2):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # Training phase
    for input_ids, attention_mask, labels in train_dataloader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_train += (predictions == labels).sum().item()
        total_train += labels.size(0)
        
    avg_loss = total_loss / len(train_dataloader)
    accuracy_train = correct_train / total_train
    
    # Print training loss and accuracy
    print(f'Epoch {epoch + 1}, Training Loss: {avg_loss:.4f}, Training Accuracy: {accuracy_train:.4f}', end='')
    
    # Validation phase
    model.eval()
    total_validation_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for input_ids, attention_mask, labels in test_dataloader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            
            validation_loss = outputs.loss
            total_validation_loss += validation_loss.item()
            
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_val += (predictions == labels).sum().item()
            total_val += labels.size(0)
    
    avg_validation_loss = total_validation_loss / len(test_dataloader)
    validation_accuracy = correct_val / total_val
    
    # Print validation loss and accuracy
    print(f', Validation Loss: {avg_validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}')
    
    # Save model if validation loss improves
    if avg_validation_loss < best_validation_loss:
        best_validation_loss = avg_validation_loss
        torch.save(model.state_dict(), f'/kaggle/working/best_classifier.pth')
 
# Save final model
torch.save(model.state_dict(), '/kaggle/working/classifierT.pth')


Epoch 1, Training Loss: 0.4354, Training Accuracy: 0.7951, Validation Loss: 0.5545, Validation Accuracy: 0.7325
Epoch 2, Training Loss: 0.4028, Training Accuracy: 0.8164, Validation Loss: 0.5915, Validation Accuracy: 0.7300


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
  
true_labels = []
predicted_labels = []
probabilities = [] 
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        _, predicted = torch.max(logits, dim=1)
        
        correct_predictions += torch.sum(predicted == labels).item()
        total_predictions += len(labels)
        
        # Store true labels and predicted labels
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted.tolist())
        
        # Store probabilities for AUC calculation
        probabilities.extend(torch.softmax(logits, dim=1)[:, 1].tolist())


accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)

precision = precision_score(true_labels, predicted_labels)
print("Precision:", precision)

recall = recall_score(true_labels, predicted_labels)
print("Recall:", recall)

f1 = f1_score(true_labels, predicted_labels)
print("F1 Score:", f1)

auc = roc_auc_score(true_labels, probabilities)
print("Area under ROC Curve (AUC):", auc)

conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.73
Precision: 0.6895874263261297
Recall: 0.47885402455661663
F1 Score: 0.5652173913043478
Area under ROC Curve (AUC): 0.7780149045289655
Confusion Matrix:
[[1109  158]
 [ 382  351]]
